The provided code is a Jupyter Notebook code snippet. Here is a summary of each part:

1. `import os`: Importing the `os` module for operating system related functionalities.

2. `%pwd`: A Jupyter magic command that retrieves the current working directory and displays it as output.

3. `os.chdir("../")`: Changing the current working directory to the parent directory.

4. `%pwd`: Retrieving the current working directory again to verify the change.

5. `from dataclasses import dataclass`: Importing the `dataclass` decorator from the `dataclasses` module.

6. `from pathlib import Path`: Importing the `Path` class from the `pathlib` module.

7. `@dataclass(frozen=True)`: Decorating the `DataIngestionConfig` class with `dataclass` and setting `frozen=True` to make it immutable.

8. `class DataIngestionConfig`: Defining the `DataIngestionConfig` class with attributes `root_dir`, `source_URL`, `local_data_file`, and `unzip_dir`.

9. `from textSummarizer.constants import *`: Importing all constants from the `textSummarizer.constants` module.

10. `from textSummarizer.utils.common import read_yaml, create_directories`: Importing the `read_yaml` and `create_directories` functions from the `textSummarizer.utils.common` module.

11. `class ConfigurationManager`: Defining the `ConfigurationManager` class responsible for managing the project's configuration.

12. The `__init__` method within `ConfigurationManager` initializes the class instance with `config_filepath` and `params_filepath` parameters and reads YAML files using the `read_yaml` function. It also creates necessary directories using the `create_directories` function.

13. `get_data_ingestion_config` method within `ConfigurationManager` retrieves the data ingestion configuration from the YAML file. It creates necessary directories and returns an instance of `DataIngestionConfig` with the configuration values.

14. `import os`: Importing the `os` module.

15. `import urllib.request as request`: Importing the `request` module from `urllib.request` for downloading files.

16. `import zipfile`: Importing the `zipfile` module for extracting zip files.

17. `from textSummarizer.logging import logger`: Importing the `logger` object from the `textSummarizer.logging` module.

18. `from textSummarizer.utils.common import get_size`: Importing the `get_size` function from the `textSummarizer.utils.common` module.

19. `class DataIngestion`: Defining the `DataIngestion` class responsible for downloading and extracting data files.

20. The `__init__` method within `DataIngestion` initializes the class instance with a `DataIngestionConfig` object.

21. The `download_file` method within `DataIngestion` downloads the file from the source URL using the `request.urlretrieve` function. It checks if the file already exists and logs the download information or the existing file's size.

22. The `extract_zip_file` method within `DataIngestion` extracts the zip file into the specified directory using `zipfile.ZipFile` and `zip_ref.extractall` functions.

23. The `try` block starts, where the configuration manager is created, the data ingestion configuration is retrieved, an instance of `DataIngestion` is created, the file is downloaded using the `download_file` method, and the zip file is extracted using the `extract_zip_file` method.

24. If any exception occurs during the execution, it is caught in the `

except` block, and the exception is raised again.

Overall, the code snippet sets up the configuration, downloads a file from a source URL, and extracts a zip file into a specified directory.

In [1]:
import os

In [2]:
%pwd

'f:\\artificial intelegnce\\study\\ML End To End Projects Krish Naik\\github\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\artificial intelegnce\\study\\ML End To End Projects Krish Naik\\github\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-19 18:17:59,559: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-19 18:17:59,568: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-19 18:17:59,571: INFO: common: created directory at: artifacts]
[2023-06-19 18:17:59,571: INFO: common: created directory at: artifacts/data_ingestion]
[2023-06-19 18:18:06,091: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F570:0F4A:122377:14E418:649071A9
Accept-Ranges: bytes
Date: Mon, 19 Jun 2023 15:18:02 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs10553-MRS
X-Cache: MIS